In [1]:
!pip install nba_api ipywidgets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.0/319.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import numpy as np

from nba_api.stats.endpoints import leaguegamelog

import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

seasons = [
    "2019-20",
    "2020-21",
    "2021-22",
    "2022-23",
    "2023-24",
    "2024-25",
    "2025-26",
]
seasons



['2019-20', '2020-21', '2021-22', '2022-23', '2023-24', '2024-25', '2025-26']

In [7]:
all_logs = []

for s in seasons:
    print("Loading season:", s)
    logs = leaguegamelog.LeagueGameLog(
        player_or_team_abbreviation="P",
        season=s,
        season_type_all_star="Regular Season"
    )
    df_season = logs.get_data_frames()[0]
    df_season["SEASON"] = s
    all_logs.append(df_season)

df_logs = pd.concat(all_logs, ignore_index=True)
df_logs["GAME_DATE"] = pd.to_datetime(df_logs["GAME_DATE"])

print("Total rows:", len(df_logs))
df_logs.head()


Loading season: 2019-20
Loading season: 2020-21
Loading season: 2021-22
Loading season: 2022-23
Loading season: 2023-24
Loading season: 2024-25
Loading season: 2025-26
Total rows: 158101


,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE,SEASON
0,22019,202324,Derrick Favors,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,...,2,0,1,1,5,6,-12,19.4,1,2019-20
1,22019,1627742,Brandon Ingram,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,...,5,1,2,2,4,22,-19,42.5,1,2019-20
2,22019,1628404,Josh Hart,1610612740,NOP,New Orleans Pelicans,0021900001,2019-10-22,NOP @ TOR,L,...,1,0,1,1,4,15,-1,30.5,1,2019-20
3,22019,101150,Lou Williams,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,...,7,1,0,2,0,21,13,38.5,1,2019-20
4,22019,201976,Patrick Beverley,1610612746,LAC,LA Clippers,0021900002,2019-10-22,LAC vs. LAL,W,...,6,0,1,2,4,2,13,24.0,1,2019-20


In [8]:
# Ensure numeric
num_cols = [
    "PTS", "REB", "AST", "FGA", "FGM",
    "FG3A", "FG3M",
    "FTA", "FTM",
    "DREB", "OREB",
    "STL", "BLK", "TOV", "PF",
]

for col in num_cols:
    df_logs[col] = pd.to_numeric(df_logs[col], errors="coerce")

df_logs["MIN"] = pd.to_numeric(df_logs["MIN"], errors="coerce")

# Composite stats
df_logs["PRA"] = df_logs["PTS"] + df_logs["REB"] + df_logs["AST"]
df_logs["PR"]  = df_logs["PTS"] + df_logs["REB"]
df_logs["PA"]  = df_logs["PTS"] + df_logs["AST"]
df_logs["RA"]  = df_logs["REB"] + df_logs["AST"]

# 3-pointers made
df_logs["THREES_MADE"] = df_logs["FG3M"]

# Blocks + steals
df_logs["BLK_STL"] = df_logs["BLK"] + df_logs["STL"]


In [9]:
# MATCHUP is like "LAL vs BOS" (home) or "LAL @ DEN" (away)
df_logs["HOME_AWAY"] = np.where(df_logs["MATCHUP"].str.contains("vs"), "Home", "Away")

# Opponent is last 3 chars (team tricode)
df_logs["OPPONENT"] = df_logs["MATCHUP"].str[-3:]

# Sort for consistency
df_logs = df_logs.sort_values(["PLAYER_ID", "GAME_DATE"]).reset_index(drop=True)

# Main working DataFrame
df = df_logs.copy()

df[["PLAYER_NAME", "SEASON", "GAME_DATE", "MATCHUP", "HOME_AWAY", "OPPONENT", "PTS"]].head()


,PLAYER_NAME,SEASON,GAME_DATE,MATCHUP,HOME_AWAY,OPPONENT,PTS
0,Vince Carter,2019-20,2019-10-24,ATL @ DET,Away,DET,0
1,Vince Carter,2019-20,2019-10-26,ATL vs. ORL,Home,ORL,2
2,Vince Carter,2019-20,2019-10-28,ATL vs. PHI,Home,PHI,4
3,Vince Carter,2019-20,2019-10-29,ATL @ MIA,Away,MIA,7
4,Vince Carter,2019-20,2019-10-31,ATL vs. MIA,Home,MIA,2


In [10]:
players_list = sorted(df["PLAYER_NAME"].unique())

metric_options = [
    ("Points", "PTS"),
    ("Rebounds", "REB"),
    ("Assists", "AST"),
    ("PRA (Pts+Reb+Ast)", "PRA"),
    ("PR (Pts+Reb)", "PR"),
    ("PA (Pts+Ast)", "PA"),
    ("RA (Reb+Ast)", "RA"),
    ("Minutes", "MIN"),
    ("FGA", "FGA"),
    ("3PA", "FG3A"),
    ("3PM", "THREES_MADE"),
    ("FTA", "FTA"),
    ("Steals", "STL"),
    ("Blocks", "BLK"),
    ("BLK+STL", "BLK_STL"),
    ("Turnovers", "TOV"),
]

season_values = sorted(df["SEASON"].unique())
season_options = [("All seasons", "All")] + [(s, s) for s in season_values]

opponent_values = sorted(df["OPPONENT"].unique())
opponent_options = [("All opponents", "All")] + [(o, o) for o in opponent_values]

len(players_list), season_values[:5], opponent_values[:5]


(1138,
 ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24'],
 ['ATL', 'BKN', 'BOS', 'CHA', 'CHI'])

In [11]:
# Player search box
player_search = widgets.Text(
    value="",
    placeholder="Type part of a name (e.g., 'luka')",
    description="Search:",
    layout=widgets.Layout(width="300px")
)

# Player dropdown
player_dropdown = widgets.Dropdown(
    options=players_list,
    description="Player:",
    layout=widgets.Layout(width="300px")
)

# Stat for over/under line
stat_dropdown = widgets.Dropdown(
    options=metric_options,
    value="PTS",
    description="Line stat:",
    layout=widgets.Layout(width="250px")
)

# Extra columns to display
metrics_display = widgets.SelectMultiple(
    options=metric_options,
    value=("PTS", "REB", "AST"),
    description="Show cols:",
    layout=widgets.Layout(width="250px", height="160px")
)

# Number of games
num_games_dropdown = widgets.Dropdown(
    options=[("Last 5", 5),
             ("Last 10", 10),
             ("Last 15", 15),
             ("Last 20", 20),
             ("All season", "All")],
    value=10,
    description="# Games:"
)

# Home / Away filter
location_dropdown = widgets.Dropdown(
    options=[("All games", "All"),
             ("Home only", "Home"),
             ("Away only", "Away")],
    value="All",
    description="Location:"
)

# Season filter
season_dropdown = widgets.Dropdown(
    options=season_options,
    value="All",
    description="Season:",
    layout=widgets.Layout(width="200px")
)

# Opponent filter
opponent_dropdown = widgets.Dropdown(
    options=opponent_options,
    value="All",
    description="Opponent:",
    layout=widgets.Layout(width="200px")
)

# Line value
line_input = widgets.FloatText(
    value=20.5,
    description="Line:",
    step=0.5,
    layout=widgets.Layout(width="200px")
)

# Output area
output = widgets.Output()


In [12]:
def on_search_change(change):
    text = change["new"].lower()
    if text == "":
        filtered = players_list
    else:
        filtered = [p for p in players_list if text in p.lower()]
    if not filtered:
        filtered = ["No match"]
    player_dropdown.options = filtered

player_search.observe(on_search_change, names="value")


In [17]:
def show_filtered_stats_and_ou(player_name, stat_col, display_metrics,
                               num_games, location, season, opponent, line_value):
    output.clear_output()
    with output:
        if player_name == "No match":
            print("No player selected. Adjust the search.")
            return

        # 1. Filter by player
        pdf = df[df["PLAYER_NAME"] == player_name].copy()
        if pdf.empty:
            print("No data for", player_name)
            return

        # 2. Season filter
        if season != "All":
            pdf = pdf[pdf["SEASON"] == season]

        if pdf.empty:
            print(f"No games found for {player_name} in season {season}.")
            return

        # 3. Opponent filter
        if opponent != "All":
            pdf = pdf[pdf["OPPONENT"] == opponent]

        if pdf.empty:
            print(f"No games vs {opponent} for {player_name} in season {season}.")
            return

        # 4. Location filter
        if location != "All":
            pdf = pdf[pdf["HOME_AWAY"] == location]

        if pdf.empty:
            print(f"No {location.lower()} games vs {opponent} for {player_name} in season {season}.")
            return

        # 5. Sort and slice last N / all
        pdf = pdf.sort_values("GAME_DATE")
        if num_games != "All":
            pdf_filtered = pdf.tail(int(num_games)).copy()
        else:
            pdf_filtered = pdf.copy()

        if pdf_filtered.empty:
            print("No games after filters.")
            return

        # 6. Stat exists?
        if stat_col not in pdf_filtered.columns:
            print(f"Stat column '{stat_col}' not found in data.")
            return

        # 7. Columns to display
        extra_cols = list(display_metrics)
        base_cols = ["GAME_DATE", "SEASON", "HOME_AWAY", "OPPONENT", "MATCHUP"]

        if stat_col not in extra_cols:
            extra_cols = [stat_col] + extra_cols

        display_cols = base_cols + extra_cols

        print(f"Showing {len(pdf_filtered)} games for {player_name}")
        display(pdf_filtered[display_cols])

        # 8. Over / Under / Push
        series = pdf_filtered[stat_col].astype(float)
        over = (series > line_value).sum()
        under = (series < line_value).sum()
        push = (series == line_value).sum()
        total = len(series)

        print(f"\nLine check for {stat_col} at {line_value}:")
        print(f"Over : {over} ({over/total:.1%})")
        print(f"Under: {under} ({under/total:.1%})")
        print(f"Push : {push} ({push/total:.1%})")
        print(f"Total: {total} games")

        # 9. Averages
        metric_cols_numeric = [c for c in extra_cols if c in pdf_filtered.columns]
        averages = pdf_filtered[metric_cols_numeric].mean().round(2)
        print("\nAverages for these filtered games:")
        for c in metric_cols_numeric:
            print(f"{c}: {averages[c]}")

                # 10. Bar chart: green if over, red if under, gray if push
        fig, ax = plt.subplots(figsize=(10, 4))

        # X positions (one per game)
        x = np.arange(len(pdf_filtered))
        values = series.values

        # Colors based on over/under/push
        colors = []
        for v in values:
            if v > line_value:
                colors.append("green")
            elif v < line_value:
                colors.append("red")
            else:
                colors.append("gray")

        # Bar chart
        ax.bar(x, values, color=colors)

        # Horizontal line for the prop line
        ax.axhline(line_value, linestyle="--", linewidth=1)

        # X-axis labels: MATCHUP on top, date underneath in mm-dd-yy
        labels = [
            f"{row.MATCHUP}\n{row.GAME_DATE.strftime('%m-%d-%y')}"
            for _, row in pdf_filtered.iterrows()
        ]

        ax.set_xticks(x)
        ax.set_xticklabels(labels, rotation=45, ha="right")

        ax.set_ylabel(stat_col)
        ax.set_title(f"{player_name} – {stat_col} vs line {line_value}")
        fig.tight_layout()
        plt.show()



In [18]:
controls_top = widgets.HBox([player_search, player_dropdown])

controls_mid = widgets.HBox([
    widgets.VBox([
        stat_dropdown,
        line_input,
        num_games_dropdown,
        location_dropdown,
        season_dropdown,
        opponent_dropdown
    ]),
    metrics_display
])

ui = widgets.VBox([controls_top, controls_mid, output])

def update_view(change=None):
    player_name = player_dropdown.value
    stat_col = stat_dropdown.value
    display_metrics = [v for v in metrics_display.value]
    num_games = num_games_dropdown.value
    location = location_dropdown.value
    season = season_dropdown.value
    opponent = opponent_dropdown.value
    line_value = line_input.value

    show_filtered_stats_and_ou(
        player_name,
        stat_col,
        display_metrics,
        num_games,
        location,
        season,
        opponent,
        line_value
    )

# Attach listeners
player_dropdown.observe(update_view, names="value")
stat_dropdown.observe(update_view, names="value")
metrics_display.observe(update_view, names="value")
num_games_dropdown.observe(update_view, names="value")
location_dropdown.observe(update_view, names="value")
season_dropdown.observe(update_view, names="value")
opponent_dropdown.observe(update_view, names="value")
line_input.observe(update_view, names="value")

display(ui)
update_view()


In [19]:
df.to_pickle("nba_games_2019_2026.pkl")